## Import libraries

In [1]:
import pandas as pd
import gc
import numpy as np
np.random.seed(0)
import json
import os
import torch
from tqdm import tqdm

import glob
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
local = True

if not local:
    from google.colab import drive
    drive.mount('/content/drive')#, force_remount=True)
    %cd /content/drive/MyDrive/Sun/ML Shock/Final project/

## Important properties

In [3]:
votes_cols = [
    "seizure_vote",
    "lpd_vote",
    "gpd_vote",
    "lrda_vote",
    "grda_vote",
    "other_vote",
]
N_classes = len(votes_cols)

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if not train_on_gpu:
    print("CUDA is not available.  Training on CPU.")
else:
    print("CUDA is available!  Training on GPU.")
    print("GPU count", torch.cuda.device_count())

if local:
    dir = "C:\\Users\\Amy\\Desktop\\Green_Git\\eegClassification\\"
    # Path to data subfolder
    path = dir+"data/data/"
    # Path to output to
    path_out = dir+"models/"
    # Path to save temporary data
    save_path = dir+"data/tmp/"
    # path to train data file
    path_df = dir+"sample_data\\"
else:
    # specify paths for google colab
    dir = "/content/drive/MyDrive/Sun/ML Shock/Final project/"
    # Path to data subfolder
    path = dir+"data/"
    # Path to output to
    path_out = dir+"models/"
    # Path to save temporary data
    save_path = dir+"data/tmp/"
    # path to train data file
    path_df = dir+"data/"


# if path does not exist create it
if not os.path.exists(path_out):
    os.makedirs(path_out)

# number of subprocesses to use for data loading
# import multiprocessing as cpu
num_workers = 0  # cpu.cpu_count()  # - 1 #
# how many samples per batch to load
batch_size = 64  # 8  #
# Is a test?
test = False  # True  #

CUDA is not available.  Training on CPU.


## Custom Dataloader

In [4]:
from torch.utils.data import Dataset
import pyarrow.parquet as pq
from scipy.signal import spectrogram

### Custom dataset for preprocessing the data

It would be time consuming to preprocess the data every time the data is loaded. Instead, I preprocess the data once and save it in a file. This way, we can load the preprocessed data directly from the file.

In [5]:
def transpose_stack(x):
    return x.transpose(0, 1).reshape(299, 400).transpose(0, 1)


def transpose_stack_eeg_spec(x):
    return x.transpose(0, 1).reshape(129, 7 * 20).transpose(0, 1)


def normalize(x):
    # Calculate mean and standard deviation once
    mean = torch.mean(x)
    std = torch.std(x)
    # Avoid reshaping multiple times
    x_flat = x.reshape(1, -1)
    # Normalize using calculated mean and std
    return (x_flat - mean) / std


def tile(x):
    return torch.tile(x, (3, 1, 1))


def normalize_special(x):
    # Use broadcasting to avoid unnecessary reshape operations
    mean = torch.tensor([0.485, 0.456, 0.406]).unsqueeze(1).unsqueeze(2)
    std = torch.tensor([0.229, 0.224, 0.225]).unsqueeze(1).unsqueeze(2)
    return (x * std - mean).reshape(3, 400, 299)


def normalize_special_eeg_spec(x):
    # Use broadcasting to avoid unnecessary reshape operations
    mean = torch.tensor([0.485, 0.456, 0.406]).unsqueeze(1).unsqueeze(2)
    std = torch.tensor([0.229, 0.224, 0.225]).unsqueeze(1).unsqueeze(2)
    return (x * std - mean).reshape(3, 7 * 20, 129)


def min_max_scaling(x):
    # Calculate min and max once
    min_val = torch.min(x, dim=1).values.unsqueeze(1)
    max_val = torch.max(x, dim=1).values.unsqueeze(1)

    # Normalize using calculated min and max
    return torch.div(x - min_val, max_val - min_val)


class CustomDataset(Dataset):
    def __init__(
        self,
        data_dir,
        data_type,
        data_info_dict,
        transform=None,
    ):
        """
        A custom data loader for the harmful brain activity dataset.
        CustomDataset inherits from torch.utils.data.Dataset
        and overrides the __len__ and __getitem__ methods.

        Parameters
        ----------
        data_dir : str
            The directory where the data is stored.
        data_type : str
            The type of data to load. Must be one of ['spec', 'eeg_raw', 'eeg_spec'].
        data_info_dict : dict
            A dictionary containing the data information. The keys are tuples of the form
            (data_id, item_id, offset) and the values are dictionaries containing the votes
            for each class.
        """

        self.data_dir = data_dir

        self.data_type = data_type

        if self.data_type not in ["eeg_spec", "spec", "eeg_raw"]:
            raise ValueError(
                "Invalid data type provided. Must be one of ['spec', 'eeg_raw', 'eeg_spec']"
            )

        if transform is None and self.data_type == "spec":
            self.transform = (
                torch.tensor,
                transpose_stack,
                normalize,
                tile,
                normalize_special,
            )
        elif transform is None and self.data_type == "eeg_spec":
            self.transform = (
                torch.tensor,
                transpose_stack_eeg_spec,
                normalize,
                tile,
                normalize_special_eeg_spec,
            )
        else:
            self.transform = transform

        self.item_list = [k for k in data_info_dict.keys()]
        self.data_info_dict = data_info_dict

    def __len__(self):
        return len(self.item_list)

    def __getitem__(self, idx):

        item = self.item_list[idx]
        data_id, _, offset = item
        path = self.data_dir + str(data_id) + ".parquet"

        data = self.preprocessing(path, offset)

        class_votes = self.data_info_dict[item]["votes"]
        if class_votes is None:
            class_votes = np.array([])
            label = np.array([])
        else:
            label = np.argmax(class_votes)

        if self.transform:
            for trans in self.transform:
                data = trans(data)

        return data, class_votes

    def preprocessing(self, path, offset):
        freq = 200 if "eeg" in self.data_type else 0.5  # Hz

        data = pq.read_table(path).to_pandas()

        # fill nan with zeros
        data = data.fillna(0)
        # enforce data type to float32
        data = data.astype(np.float32)

        collected_data = []
        if self.data_type == "spec":
            for spec_type in ["LL", "RL", "LP", "RP"]:
                collected_data.append(data.filter(regex=spec_type).values)
        elif self.data_type in ["eeg_raw", "eeg_spec"]:
            for col in data.columns:
                collected_data.append(data[col].values)

        data = np.moveaxis(np.array(collected_data), 0, -1)

        if self.data_type == "spec":
            start = int((0 + offset) * freq)
            end = int((600 + offset) * freq) - 1

            # clip values between exp(-4) and exp(8)
            data = np.clip(data[start:end, :], np.exp(-4), np.exp(8))
            data = np.log(data)
            # move last axis to first
            data = np.moveaxis(data, -1, 0)
        elif self.data_type == "eeg_raw":

            # # select the central 50 seconds of the data
            # start = int((0 + offset) * freq)
            # end = int((49 + offset) * freq)
            # select the central 10 seconds of the data
            start = int((20 + offset) * freq)
            end = int((29 + offset) * freq)

            data = data[start:end]
            # move last axis to first
            data = np.moveaxis(data, -1, 0)
        elif self.data_type == "eeg_spec":

            # # select the central 50 seconds of the data
            # start = int((0 + offset) * freq)
            # end = int((49 + offset) * freq)
            # select the central 10 seconds of the data
            start = int((20 + offset) * freq)
            end = int((29 + offset) * freq)

            data = data[start:end]
            # apply spectrogram to across all channels, axis 0
            spec_data = []
            for d in range(data.shape[1]):
                _, _, Sxx = spectrogram(data[:, d], fs=freq)
                # clip
                Sxx = np.clip(Sxx, np.exp(-4), np.exp(8))
                spec_data.append(np.log(Sxx))

            data = np.moveaxis(np.array(spec_data), 0, -1)
            data = np.array(spec_data)
        else:
            raise ValueError(
                "Invalid data type provided. Must be one of ['spec', 'eeg_raw', 'eeg_spec']"
            )

        return data

### Custom dataset for loading the saved preprocessed data

In [6]:
class CustomDatasetNPY(Dataset):
    def __init__(
        self,
        data_path,
        data_files,
        transform=None,
    ):
        self.data_path = data_path
        self.data_files = data_files
        self.N_items = len(self.data_files)
        self.transform = transform

    def __len__(self):
        return self.N_items

    def __getitem__(self, idx):

        idx = self.data_files[idx]
        # read data from path, npy
        data = np.load(self.data_path + "images_" + str(idx) + ".npy")
        class_votes = np.load(self.data_path + "votes_" + str(idx) + ".npy")

        if self.transform:
            batch_size = data.shape[0]
            for trans in self.transform:
                data = trans(batch_size, data)

        return data, class_votes

## Model Architectures

In [7]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

# define the CNN architecture
class CustomCNN(nn.Module):
    def __init__(self, input_shape, N_classes, batch_size=64):
        super(CustomCNN, self).__init__()

        self.input_shape = input_shape
        self.N_classes = N_classes
        self.batch_size = batch_size

        self.conv1 = nn.Conv2d(self.input_shape[0], 8, 3, padding=1)
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1)
        self.conv3 = nn.Conv2d(8, 8, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        self.N_out = (
            8
            * (self.input_shape[1] // 2 // 2 // 2)
            * (self.input_shape[2] // 2 // 2 // 2)
        )
        self.fc1 = nn.Linear(self.N_out, 500)
        # linear layer (500 -> 6)
        self.fc2 = nn.Linear(500, self.N_classes)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(x.shape[0], self.N_out)  # x.view(self.batch_size, self.N_out)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = F.log_softmax(self.fc2(x), dim=1)

        return x


# define the CNN architecture
class CustomCNN_eeg(nn.Module):
    def __init__(self, input_shape, N_classes, batch_size=64):
        super(CustomCNN_eeg, self).__init__()

        self.input_shape = input_shape
        self.N_classes = N_classes
        self.batch_size = batch_size

        self.conv1 = nn.Conv2d(self.input_shape[0], 8, 3, padding=1)
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1)
        self.conv3 = nn.Conv2d(8, 8, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        self.N_out = (
            8
            * (self.input_shape[1] // 2 // 2 // 2)
            * (self.input_shape[2] // 2 // 2 // 2)
        )
        self.fc1 = nn.Linear(self.N_out, 500)
        # linear layer (500 -> 6)
        self.fc2 = nn.Linear(500, self.N_classes)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(x.shape[0], self.N_out)  # x.view(self.batch_size, self.N_out)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = F.log_softmax(self.fc2(x), dim=1)

        return x


class CustomCNN_eeg_small(nn.Module):
    def __init__(self, input_shape, N_classes, batch_size=64):
        super(CustomCNN_eeg_small, self).__init__()

        self.input_shape = input_shape
        self.N_classes = N_classes
        self.batch_size = batch_size

        self.conv1 = nn.Conv2d(self.input_shape[0], 4, 4, padding=1)
        self.conv2 = nn.Conv2d(4, 8, 3, padding=1)
        self.conv3 = nn.Conv2d(8, 8, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        self.N_out = (
            8
            * (self.input_shape[1] // 2 // 2 // 2)
            * (self.input_shape[2] // 2 // 2 // 2)
        )
        self.fc1 = nn.Linear(self.N_out, 500)
        # linear layer (500 -> 6)
        self.fc2 = nn.Linear(500, self.N_classes)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(x.shape[0], self.N_out)  # x.view(self.batch_size, self.N_out)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = F.log_softmax(self.fc2(x), dim=1)

        return x


# define the CNN architecture
class TransNet_Resnet18(nn.Module):
    def __init__(self, input_shape, N_classes):
        super(TransNet_Resnet18, self).__init__()

        self.input_shape = input_shape

        self.N_classes = N_classes

        self.model = models.resnet18(pretrained=True)

        in_features = self.model.fc.in_features

        # freeze all model parameters
        for param in self.model.parameters():
            param.requires_grad = False

        # remove the last layer
        self.model.fc = nn.Identity()

        # a layer to go some shape (4,299,100) to (3,299,100)
        # self.conv1 = nn.Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        # replace first layer with new layer
        # self.model.conv1 = self.conv1

        # add new layer
        # self.fc = nn.Linear(in_features, self.N_classes)

    def forward(self, x):
        x = self.model(x)
        # x = F.log_softmax(self.fc(x), dim=1)

        return x


# define the CNN architecture
class TransNet_Efficientnetb0(nn.Module):
    def __init__(self, input_shape, N_classes):
        super(TransNet_Efficientnetb0, self).__init__()

        self.N_classes = N_classes

        self.model = models.efficientnet_b0(pretrained=True)
        in_features = self.model.classifier[-1].in_features

        # freeze all model parameters
        for param in self.model.parameters():
            param.requires_grad = False

        # # remove the last layer
        self.model.classifier[-1] = nn.Identity()

        # a layer to go some shape (4,299,100) to (3,299,100)
        # self.conv1 = nn.Conv2d(4, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        # replace first layer with new layer
        # self.model.features[0] = self.conv1

        # add new layer
        # self.fc = nn.Linear(in_features, self.N_classes)

    def forward(self, x):
        x = self.model(x)
        # x = F.log_softmax(self.fc(x), dim=1)

        return x


class TransNet_Resnet18_unfrozen(nn.Module):
    def __init__(self, input_shape, N_classes):
        super(TransNet_Resnet18_unfrozen, self).__init__()

        self.input_shape = input_shape

        self.N_classes = N_classes

        self.model = models.resnet18(pretrained=True)

        in_features = self.model.fc.in_features

        # freeze all model parameters
        # for param in self.model.parameters():
        #     param.requires_grad = False

        # remove the last layer
        self.model.fc = nn.Identity()

        # a layer to go some shape (4,299,100) to (3,299,100)
        # self.conv1 = nn.Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        # replace first layer with new layer
        # self.model.conv1 = self.conv1

        # add new layer
        self.fc = nn.Linear(in_features, self.N_classes)

    def forward(self, x):
        x = self.model(x)
        x = F.log_softmax(self.fc(x), dim=1)

        return x


# define the CNN architecture
class TransNet_Efficientnetb0_unfrozen(nn.Module):
    def __init__(self, input_shape, N_classes):
        super(TransNet_Efficientnetb0_unfrozen, self).__init__()

        self.N_classes = N_classes

        self.model = models.efficientnet_b0(pretrained=True)
        in_features = self.model.classifier[-1].in_features

        # freeze all model parameters
        # for param in self.model.parameters():
        #     param.requires_grad = False

        # # remove the last layer
        self.model.classifier[-1] = nn.Identity()

        # a layer to go some shape (4,299,100) to (3,299,100)
        # self.conv1 = nn.Conv2d(4, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        # replace first layer with new layer
        # self.model.features[0] = self.conv1

        # add new layer
        self.fc = nn.Linear(in_features, self.N_classes)

    def forward(self, x):
        x = self.model(x)
        x = F.log_softmax(self.fc(x), dim=1)

        return x



## Helper functions

In [8]:
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix

In [9]:

# all the model architectures
# module = __import__("model_architectures")


get_batch_transform = lambda x, y: (
    x[0, :],
    y[0, :],
)


def get_data_info(df, data_type):

    # train is true id votes_cols are available
    train = all([col in df.columns for col in votes_cols])

    label_cols = (
        ["eeg_id", "label_id", "eeg_label_offset_seconds"]
        if "eeg" in data_type
        else ["spectrogram_id", "label_id", "spectrogram_label_offset_seconds"]
    )
    offset = (
        ["eeg_label_offset_seconds"]
        if "eeg" in data_type
        else ["spectrogram_label_offset_seconds"]
    )

    # if info_cols not in df add it and set to zero
    for col in offset:
        if col not in df.columns:
            df[col] = 0
    # if df does not contain "label_id" add a unique label_id
    if "label_id" not in df.columns:
        df["label_id"] = range(len(df))

    info = {}
    df_gr = df.groupby(label_cols)
    for name, group in df_gr:
        # first row of group
        info[name] = {"votes": group[votes_cols].values[0] if train else None}

    return info


def shuffle(info):
    info_shuffled = {}
    keys = [k for k in info.keys()]
    np.random.shuffle(keys)
    for key in keys:
        info_shuffled[key] = info[key]

    return info_shuffled


def egg_spec_augmentation(info, rates=[0.75, 0.2, 0.2, 0.2, 0.2, 0.2]):

    info_aug = info.copy()

    offset = [-1, 1, -2, 2, -3, 3, -4, 4]
    keys = [k for k in info_aug.keys()]
    N_item = len(keys)

    counts = [0] * 6
    for k in info_aug.keys():
        idx = np.argmax(info_aug[k]["votes"])
        counts[idx] += 1

    for key in keys:
        # generate random between 0 and 1
        r = np.random.rand()
        idx = np.argmax(info_aug[key]["votes"])

        if r < rates[idx]:
            # select a random offset
            off = np.random.choice(offset)
            new_key = (key[0], key[1], key[2] + off)
            # if new_key not in info add it
            if new_key not in info_aug.keys():
                info_aug[new_key] = info_aug[key]

            counts[idx] += 1

        # if len(info) >= N_item*(1+0.4):
        #     break

    return info_aug


def lrfn(
    epoch,
    epochs,
    mode="cos",
    lr_start=2e-4,
    lr_max=3e-5 * 64,
    lr_min=1e-5,
    lr_ramp_ep=4,
    lr_sus_ep=0,
    lr_decay=0.75,
):
    if epoch < lr_ramp_ep:
        lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
    elif epoch < lr_ramp_ep + lr_sus_ep:
        lr = lr_max
    elif mode == "exp":
        lr = (lr_max - lr_min) * lr_decay ** (epoch - lr_ramp_ep - lr_sus_ep) + lr_min
    elif mode == "step":
        lr = lr_max * lr_decay ** ((epoch - lr_ramp_ep - lr_sus_ep) // 2)
    elif mode == "cos":
        decay_total_epochs, decay_epoch_index = (
            epochs - lr_ramp_ep - lr_sus_ep + 3,
            epoch - lr_ramp_ep - lr_sus_ep,
        )
        phase = np.pi * decay_epoch_index / decay_total_epochs
        lr = (lr_max - lr_min) * 0.5 * (1 + np.cos(phase)) + lr_min
    return lr


def train_func(
    model,
    train_loader,
    valid_loader,
    path_model_out,
    n_epochs,
    learning_rate,
    label_smoothing,
    test=False,
    valid_loss_min=np.Inf,
    loss_type = "cross_entropy",
):
    
    if loss_type == "cross_entropy":
        criterion = nn.CrossEntropyLoss()
    elif loss_type == "kldiv":
        criterion = nn.KLDivLoss(reduction="batchmean", log_target=True)
    else:
        raise ValueError("Loss not found")
    
    optimizer = optim.Adam(model.parameters())

    track_loss = []
    track_loss_val = []

    max_samples = 1

    train_on_gpu = torch.cuda.is_available()

    for epoch in range(1, n_epochs + 1):

        for g in optimizer.param_groups:
            g["lr"] = learning_rate[epoch - 1]  # lrfn(epoch, n_epochs)
        # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        model.train()
        count = 0

        for data, votes in tqdm(train_loader):
            if test and count >= max_samples:
                break

            data, votes = get_batch_transform(data, votes)
            # offset vote by adding label smoothing as offset
            votes = votes * (1 - label_smoothing) + label_smoothing / N_classes

            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, votes = data.cuda(), votes.cuda()
            # clear the gradients of all optimized variables
            optimizer.zero_grad()

            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)

            if torch.isnan(output).sum() > 0:
                print("Nan in output")
            else:

                # loss
                if loss_type == "cross_entropy":
                    loss = criterion(torch.exp(output), torch.argmax(votes, axis=1))
                elif loss_type == "kldiv":
                    loss = criterion(output.float(), F.log_softmax(votes.float(), dim=1))
                else:
                    raise ValueError("Loss not found")

                # update training loss
                train_loss += loss.item()  # *data.size(0)

                # backward pass: compute gradient of the loss with respect to model parameters
                loss.backward()

                optimizer.step()

                count += 1

        train_loss = train_loss / count  # len(train_loader)#/batch_size

        torch.cuda.empty_cache()
        ######################
        # validate the model #
        ######################
        model.eval()
        count = 0
        for data, votes in tqdm(valid_loader):
            if test and count >= max_samples:
                break

            data, votes = get_batch_transform(data, votes)
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, votes = data.cuda(), votes.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)

            if torch.isnan(output).sum() > 0:
                print("Nan in output")
            else:
                # calculate the batch loss
                if loss_type == "cross_entropy":
                    loss = criterion(torch.exp(output), torch.argmax(votes, axis=1))
                elif loss_type == "kldiv":
                    loss = criterion(output.float(), F.log_softmax(votes.float(), dim=1))
                else:
                    raise ValueError("Loss not found")

                # update average validation loss
                valid_loss += loss.item()  # *data.size(0)

                count += 1

        torch.cuda.empty_cache()
        # calculate average losses

        valid_loss = valid_loss / count  # len(valid_loader)#/batch_size

        track_loss.append(train_loss)
        track_loss_val.append(valid_loss)

        # print training/validation statistics
        print("{}; \t{:.6f}; \t{:.6f}".format(epoch, train_loss, valid_loss))

        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            torch.save(model.state_dict(), path_model_out)
            print(
                "Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...".format(
                    valid_loss_min, valid_loss
                )
            )
            valid_loss_min = valid_loss

        gc.collect()

    return track_loss, track_loss_val


def create_data_loaders(
    path,
    df,
    train_p,
    test_p,
    data_type,
    min_votes,
    augmentation,
    batch_size=64,
    num_workers=0,
):
    data_dir = f"train_eegs/" if "eeg" in data_type else f"train_spectrograms/"
    data_dir = path + data_dir

    df_test = df[df["patient_id"].isin(test_p)]
    print("Fetching info...")
    info_test = get_data_info(df_test, data_type)

    dataset_test = CustomDataset(data_dir, data_type, info_test)
    test_loader = DataLoader(
        dataset_test, batch_size=batch_size, shuffle=False, num_workers=num_workers
    )

    # split train_p into train and validation
    split = int(0.8 * len(train_p))
    train_p, valid_p = train_p[:split], train_p[split:]

    df_train = df[df["patient_id"].isin(train_p)]
    df_train = df_train[df_train["total_votes"] >= min_votes]
    info_train = get_data_info(df_train, data_type)
    info_train = shuffle(info_train)

    if augmentation:
        info_train_aug = egg_spec_augmentation(info_train)
    else:
        info_train_aug = info_train.copy()

    df_valid = df[df["patient_id"].isin(valid_p)]
    # df_valid = df_valid[df_valid["total_votes"] >= min_votes]
    info_valid = get_data_info(df_valid, data_type)
    info_valid = shuffle(info_valid)

    print("Creating data loaders...")
    train_dataset = CustomDataset(data_dir, data_type, info_train_aug)
    valid_dataset = CustomDataset(data_dir, data_type, info_valid)

    # prepare data loaders (combine dataset and sampler)
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, num_workers=num_workers
    )
    valid_loader = DataLoader(
        valid_dataset, batch_size=batch_size, num_workers=num_workers
    )

    return train_loader, valid_loader, test_loader


def run(
    path_out,
    model_name,
    label_smoothing,
    input_shape,
    n_epochs,
    model_info,
    train_loader,
    valid_loader,
    test_loader,
    is_test=False,
    loss_type="cross_entropy",
):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_on_gpu = torch.cuda.is_available()

    # get model name as executable function
    # model = getattr(module, model_name)(input_shape=input_shape, N_classes=N_classes)
    model = globals()[model_name](input_shape=input_shape, N_classes=N_classes)


    if train_on_gpu:
        model.cuda()

    num_parameters = sum(p.numel() for p in model.parameters())
    # print("Number of parameters in the model", num_parameters)
    model_info["num_parameters"] = num_parameters

    # number of trainable parameters
    num_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    # print("Number of trainable parameters in the model", num_parameters)
    model_info["num_trainable_parameters"] = num_parameters

    # check if "f"./model_{model_name}_*" exists and add 1 to the index
    index = 0
    model_path = path_out + "model_{model_name}_{index}.pt"
    while os.path.exists(model_path.format(model_name=model_name, index=index)):
        index += 1

    configs = {
        "n_epochs": n_epochs,
        "path_model_out": path_out + f"model_{model_name}_{index}.pt",
        "learning_rate": [lrfn(epoch, n_epochs) for epoch in np.arange(n_epochs)],
        "label_smoothing": label_smoothing,
        "loss_type": loss_type,
    }

    model_info["configs"] = configs
    model_info["model_name"] = model_name
    model_info["input_shape"] = input_shape

    valid_loss_min = (
        np.min(model_info["track_loss_val"])
        if len(model_info["track_loss_val"]) > 0
        else np.Inf
    )

    track_loss, track_loss_val = train_func(
        model,
        train_loader,
        valid_loader,
        valid_loss_min=valid_loss_min,
        test=is_test,
        **configs,
    )

    model_info["track_loss"] += track_loss
    model_info["track_loss_val"] += track_loss_val

    model.load_state_dict(torch.load(configs["path_model_out"]))

    if loss_type == "cross_entropy":
        criterion = nn.CrossEntropyLoss()
    elif loss_type == "kldiv":
        criterion = nn.KLDivLoss(reduction="batchmean", log_target=True)
    else:
        raise ValueError("Loss not found")


    if test_loader is not None:

        # track test loss
        test_loss = 0.0
        test_loss_baseline = 0.0
        class_correct = list(0.0 for i in range(N_classes))
        class_total = list(0.0 for i in range(N_classes))
        model.eval()

        max_samples = 1
        cm_y_pred = []
        cm_y_true = []
        # iterate over test data
        count = 0
        for data, votes in tqdm(test_loader):
            if is_test and count >= max_samples:
                break

            data, votes = get_batch_transform(data, votes)
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, votes = data.cuda(), votes.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)

            if torch.isnan(output).sum() > 0:
                print("Nan in output")
            else:
                count += 1
                # calculate the batch loss
                if loss_type == "cross_entropy":
                    loss = criterion(torch.exp(output), torch.argmax(votes, axis=1))
                elif loss_type == "kldiv":
                    loss = criterion(output.float(), F.log_softmax(votes.float(), dim=1))
                else:
                    raise ValueError("Loss not found")

                test_loss += loss.item()  # *data.size(0)

                # dummy is a tensor filled with 1/6 of shape [64,6]
                dummy = torch.ones(data.size(0), N_classes).to(device)
                dummy = dummy / N_classes
                if loss_type == "cross_entropy":
                    loss_baseline = criterion(dummy, torch.argmax(votes, axis=1))
                elif loss_type == "kldiv":
                    loss_baseline = criterion(
                        F.log_softmax(dummy, dim=1), F.log_softmax(votes.float(), dim=1)
                    )
                else:
                    raise ValueError("Loss not found")

                test_loss_baseline += loss_baseline.item()

                # convert output probabilities to predicted class
                _, pred = torch.max(output, 1)
                # compare predictions to true label
                target = torch.argmax(votes, axis=1)
                correct_tensor = pred.eq(target.data.view_as(pred))
                correct = (
                    np.squeeze(correct_tensor.numpy())
                    if not train_on_gpu
                    else np.squeeze(correct_tensor.cpu().numpy())
                )
                # calculate test accuracy for each object class
                for i in range(target.shape[0]):
                    label = target.data[i]
                    class_correct[label] += correct[i].item()
                    class_total[label] += 1

                    cm_y_pred.append(pred[i].item())
                    cm_y_true.append(target.data[i].item())

        # average test loss
        test_loss = test_loss / count  # len(test_loader.dataset)  # /batch_size
        print("Test Loss: {:.6f}\n".format(test_loss))

        test_loss_baseline = (
            test_loss_baseline / count
        )  # len(test_loader.dataset)  # /batch_size
        print("Test Loss Baseline: {:.6f}\n".format(test_loss_baseline))

        cm = confusion_matrix(cm_y_true, cm_y_pred)
        cm_p = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

        model_info["test_loss"] = test_loss
        model_info["test_loss_baseline"] = test_loss_baseline
        model_info["confusion_matrix"] = cm.tolist()
        model_info["confusion_matrix_p"] = cm_p.tolist()


def save_data(loaders, names, save_path, test=False):
    for data_loader, text in zip(
        loaders,
        names,
    ):
        tmp_path = save_path + f"{text}/"
        if not os.path.exists(tmp_path):
            os.makedirs(tmp_path)

        count = 0
        for X, votes in tqdm(data_loader, desc=f"Saving {text} data"):

            if test and count >= 3:
                break

            # save the images
            np.save(tmp_path + f"images_{count}.npy", X.numpy())
            # save the votes
            np.save(tmp_path + f"votes_{count}.npy", votes.numpy())
            count += 1


def run_inference(model_name, path_model_out, test_loader, input_shape, is_test=False):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_on_gpu = torch.cuda.is_available()

    # get model name as executable function
    # model = getattr(module, model_name)(input_shape=input_shape, N_classes=N_classes)
    model = globals()[model_name](input_shape=input_shape, N_classes=N_classes)
    # load trained model
    model.load_state_dict(torch.load(path_model_out))

    criterion = nn.KLDivLoss(reduction="batchmean", log_target=True)
    # criterion = nn.CrossEntropyLoss()

    # track test loss
    test_loss = 0.0
    test_loss_baseline = 0.0
    class_correct = list(0.0 for i in range(N_classes))
    class_total = list(0.0 for i in range(N_classes))
    model.eval()

    max_samples = 1
    cm_y_pred = []
    cm_y_true = []
    predictions = []
    labels = []
    # iterate over test data
    count = 0
    for data, votes in tqdm(test_loader):
        if is_test and count >= max_samples:
            break

        data, votes = get_batch_transform(data, votes)
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, votes = data.cuda(), votes.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)

        if torch.isnan(output).sum() > 0:
            print("Nan in output")
        else:
            count += 1
            # calculate the batch loss
            loss = criterion(output.float(), F.log_softmax(votes.float(), dim=1))
            # loss = criterion(torch.exp(output), torch.argmax(votes, axis=1))

            test_loss += loss.item()  # *data.size(0)

            # dummy is a tensor filled with 1/6 of shape [64,6]
            dummy = torch.ones(data.size(0), N_classes).to(device)
            dummy = dummy / N_classes
            loss_baseline = criterion(
                F.log_softmax(dummy, dim=1), F.log_softmax(votes.float(), dim=1)
            )
            # loss_baseline = criterion(dummy, torch.argmax(votes, axis=1))

            test_loss_baseline += loss_baseline.item()

            # convert output probabilities to predicted class
            _, pred = torch.max(output, 1)
            # compare predictions to true label
            target = torch.argmax(votes, axis=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            correct = (
                np.squeeze(correct_tensor.numpy())
                if not train_on_gpu
                else np.squeeze(correct_tensor.cpu().numpy())
            )
            # calculate test accuracy for each object class
            for i in range(target.shape[0]):
                label = target.data[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1

                cm_y_pred.append(pred[i].item())
                cm_y_true.append(target.data[i].item())

        predictions.append(output.cpu().detach().numpy())
        labels.append(votes.cpu().detach().numpy())

    predictions = np.concatenate(predictions, axis=0)
    labels = np.concatenate(labels, axis=0)

    accuracy = 100 * np.sum(class_correct) / np.sum(class_total)
    print("Test Accuracy: {:.6f}%\n".format(accuracy))

    # average test loss
    test_loss = test_loss / count  # len(test_loader.dataset)  # /batch_size
    print("Test Loss: {:.6f}\n".format(test_loss))

    test_loss_baseline = (
        test_loss_baseline / count
    )  # len(test_loader.dataset)  # /batch_size
    print("Test Loss Baseline: {:.6f}\n".format(test_loss_baseline))

    cm = confusion_matrix(cm_y_true, cm_y_pred)
    cm_p = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    return accuracy, test_loss, test_loss_baseline, cm, cm_p, predictions, labels



### Get the training data info and load or create the data splits

In [10]:
df = pd.read_csv(path_df + f"train.csv")

df["total_votes"] = df[
    ["seizure_vote", "lpd_vote", "gpd_vote", "lrda_vote", "grda_vote", "other_vote"]
].sum(axis=1)

# if path_out + "split.json" exist load it
if os.path.exists(path_out + "split.json"):
    with open(path_out + "split.json", "r") as f:
        split = json.load(f)
    train_p = np.array(split["train"])
    test_p = np.array(split["test"])
    print("Loaded train/test split")
else:
    p_id = df["patient_id"].unique()
    np.random.shuffle(p_id)
    # test train split
    train_p = p_id[: int(0.8 * len(p_id))]
    test_p = p_id[int(0.8 * len(p_id)) :]
    print("Created train/test split")

    # record the split in dict and save to json
    split = {"train": train_p.tolist(), "test": test_p.tolist()}
    with open(path_out + "split.json", "w") as f:
        json.dump(split, f)

Loaded train/test split


### Different training configurations

In [11]:
# # filter data, eg number of votes >5
# min_votes = [0, 5]
# # augment data (window shifting)
# augmentation = [False, True]
# # label smoothing
# label_smoothing = [0, 0.01]
# # number of epochs to train the model
# n_epochs = 40 if not test else 1
# # data type
# data_type = "eeg_spec"  
# # model name
# model_name = "CustomCNN_eeg"
# input_shape = (3, 140, 129)
# transform = tuple()

# # filter data for #votes >5
# min_votes = [0, 5]
# # augment data
# augmentation = [False]
# # label smoothing
# label_smoothing = [0, 0.01]
# # number of epochs to train the model
# n_epochs = 20 if not test else 1
# # data type
# data_type = "spec"  
# model_name = "CustomCNN"
# input_shape = (3, 400, 299)

# filter data, eg number of votes >5
min_votes = [0, 5]
# augment data (window shifting)
augmentation = [True, False]
# label smoothing
label_smoothing = [0, 0.01]
# number of epochs to train the model
n_epochs = 10 if not test else 1
# data type
data_type = "eeg_spec"  
# model name
model_name = "CustomCNN_eeg_small"
input_shape = (1, 140, 129)
# select one of the 3 channels
transform = (lambda batch_size, x: x[:, 0, :, :].reshape(batch_size, *input_shape),)


# # filter data, eg number of votes >5
# min_votes = [0, 5]
# # augment data (window shifting)
# augmentation = [False, True]
# # label smoothing
# label_smoothing = [0, 0.01]
# # number of epochs to train the model
# n_epochs = 8 if not test else 1
# # data type
# data_type = "eeg_spec"  # "eeg_raw" #"spec" #
# # model name
# model_name = "TransNet_Resnet18_unfrozen"
# input_shape = (3, 140, 129)
# transform = tuple()

### Run the training loop for the different configurations

In [12]:
data_type

'eeg_spec'

In [13]:
first = True
for mv in min_votes:
    for aug in augmentation:

        train_loader, valid_loader, test_loader = create_data_loaders(
            path, df, train_p, test_p, data_type, mv, aug, batch_size, num_workers
        )

        if first:
            save_data(
                [train_loader, valid_loader, test_loader],
                [f"train_{data_type}", f"valid_{data_type}", f"test_{data_type}"],
                save_path,
                test,
            )
            first = False
        else:
            save_data([train_loader, valid_loader], [f"train_{data_type}", f"valid_{data_type}"], save_path, test)

        train_data = CustomDatasetNPY(
            save_path + f"train_{data_type}/",
            [str(i) for i in range(len(train_loader))],
            transform=transform,
        )
        train_loader = DataLoader(
            train_data, batch_size=1, shuffle=False, num_workers=num_workers
        )

        valid_data = CustomDatasetNPY(
            save_path + f"valid_{data_type}/",
            [str(i) for i in range(len(valid_loader))],
            transform=transform,
        )
        valid_loader = DataLoader(
            valid_data, batch_size=1, shuffle=False, num_workers=num_workers
        )

        # test_data = CustomDatasetNPY(
        #     save_path + f"test_{data_type}/",
        #     [str(i) for i in range(len(test_loader))],
        #     transform=transform,
        # )
        # test_loader = DataLoader(
        #     test_data, batch_size=1, shuffle=False, num_workers=num_workers
        # )

        test_loader = None

        for ls in label_smoothing:
            model_info = {}
            model_info["track_loss"] = []
            model_info["track_loss_val"] = []
            model_info["min_votes"] = mv
            model_info["augmentation"] = aug
            model_info["data_type"] = data_type

            run(
                path_out,
                model_name,
                ls,
                input_shape,
                n_epochs,
                model_info,
                train_loader,
                valid_loader,
                test_loader,
                is_test=test,
            )

            gc.collect()
            torch.cuda.empty_cache()

            # save model_info as json
            model_info_path = model_info["configs"]["path_model_out"].replace(
                ".pt", ".json"
            )
            with open(model_info_path, "w") as f:
                json.dump(model_info, f)

            print("Done")
            
        # permanently delete all files and folders in the train and valid tmp data folder
        for folder in [f"train_{data_type}", f"valid_{data_type}"]:
            for file in os.listdir(save_path + folder):
                os.remove(save_path + folder + file)



Fetching info...
Creating data loaders...


Saving train_eeg_spec data:   8%|▊         | 117/1395 [02:13<27:23,  1.29s/it]

In [ ]:
train_loader, valid_loader, test_loader = create_data_loaders(
            path, df, train_p, test_p, data_type, 0, False, batch_size, num_workers
)
save_data(
    [train_loader, valid_loader, test_loader],
    [f"train_{data_type}", f"valid_{data_type}", f"test_{data_type}"],
    save_path,
    test,
)

### Load performance metrics and plot loss

In [ ]:
# final all json files of the form path_out + "model_*_*.pt"
model_info_paths = glob.glob(path_out + "model_*_*.json")
print(f"Found {len(model_info_paths)} models")

In [ ]:
for index, file in enumerate(model_info_paths):
    with open(file, "r") as f:
        model_info = json.load(f)
    
    epochs = len(model_info["track_loss"])
    # plot "track_loss" and "track_loss_val"
    fig, ax = plt.subplots()
    ax.plot([i for i in range(epochs)], model_info["track_loss"], label="train")
    ax.plot([i for i in range(epochs)], model_info["track_loss_val"], label="valid")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.set_title(f"Loss: {model_info['model_name']} {index}")
    ax.legend()

    plt.show()

### Inference with the trained model

In [ ]:
with open(path_out + "split.json", "r") as f:
    split = json.load(f)
train_p = np.array(split["train"])
test_p = np.array(split["test"])

df = pd.read_csv(path_df + f"train.csv")

### Move validation data to the training data folder for inference on the full dataset

In [ ]:
# # move all files from save_path + "valid_data" to save_path + "train"
# import shutil

# valid_files = glob.glob(save_path + "valid/*")
# for file in valid_files:
#     shutil.move(file, save_path + "train/")

### Inference and evaluation on the test set

In [ ]:
for index in range(len(model_info_paths)):
    model_info_path = model_info_paths[index]

    #load model info from json
    with open(model_info_path, "r") as f:
        model_info = json.load(f)

    date_type = model_info["data_type"]
    input_shape = model_info["input_shape"]
    model_name = model_info["model_name"]
    if model_name == "CustomCNN_eeg_small":
        transform = (lambda batch_size, x: x[:, 0, :, :].reshape(batch_size, *input_shape),)
    else:
        transform = tuple()


    N_samples = len(df[df["patient_id"].isin(test_p)])
    N_samples = np.ceil(N_samples / batch_size).astype(int)
    test_data = CustomDatasetNPY(
        save_path + f"test_{data_type}/",
        [str(i) for i in range(N_samples)],
        transform=transform,
    )
    test_loader = DataLoader(
        test_data, batch_size=1, shuffle=False, num_workers=num_workers
    )

    path_model = path_out + f"model_{model_name}_{index}.pt"

    print(f"Running inference for {model_name}_{index} on test set...")
    accuracy, test_loss, test_loss_baseline, cm, cm_p, predictions, labels = (
        run_inference(model_name, path_model, test_loader, input_shape, is_test=test)
    )

    #####

    # save results

    loss_dict = {
        "test_loss_kl": test_loss,
        "test_loss_baseline_kl": test_loss_baseline,
        "test_accuracy": accuracy,
    }

    # save predictions and labels as npy
    np.save(path_out + f"test_predictions_{model_name}_{index}.npy", predictions)
    np.save(path_out + f"test_labels_{model_name}_{index}.npy", labels)

    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xticks(ticks=np.arange(6) + 0.5, labels=votes_cols)
    plt.yticks(ticks=np.arange(6) + 0.5, labels=votes_cols)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix: {model_name}_{index}")

    # plt.savefig(path_out + f"cm_{model_name}_{index}.png")

    plt.figure(figsize=(10, 10))
    sns.heatmap(cm_p * 100, annot=True, fmt=".1f", cmap="Blues")
    plt.xticks(ticks=np.arange(6) + 0.5, labels=votes_cols)
    plt.yticks(ticks=np.arange(6) + 0.5, labels=votes_cols)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix: {model_name}_{index}")

    # plt.savefig(path_out + f"cm_probs_{model_name}_{index}.png")

    plt.show()

    # print accuracy, test_loss, test_loss_baseline
    print(f"Accuracy: {accuracy}")
    print(f"Test Loss: {test_loss}")
    print(f"Test Loss Baseline: {test_loss_baseline}")


    # Inference on training set
    N_samples = len(os.listdir(save_path + f"train_{data_type}/"))//2
    train_data = CustomDatasetNPY(
        save_path + f"train_{data_type}/",
        [str(i) for i in range(N_samples)],
        transform=transform,
    )

    train_loader = DataLoader(
        train_data, batch_size=1, shuffle=False, num_workers=num_workers
    )

    print(f"Running inference for {model_name}_{index} on train set...")
    _, _, _, _, _, predictions, labels = (
        run_inference(model_name, path_model, train_loader, input_shape, is_test=test)
    )

    # save predictions and labels as npy
    np.save(path_out + f"train_predictions_{model_name}_{index}.npy", predictions)
    np.save(path_out + f"train_labels_{model_name}_{index}.npy", labels)


    # save loss_dict to json
    with open(path_out + f"eval_{model_name}_{index}.json", "w") as f:
        json.dump(loss_dict, f)

### Ensemble model

In [ ]:
data_files = os.listdir(path_out)
data_features = [f for f in data_files if f.startswith('train_predictions')]
data_votes = [f for f in data_files if f.startswith('train_labels')]

N_items = len(data_features)

print("Number of items", N_items)

test_data_features = [f for f in data_files if f.startswith('test_predictions')]
test_data_votes = [f for f in data_files if f.startswith('test_labels')]

N_test_items = len(test_data_features)

print("Number of test items", N_test_items)

In [ ]:
# load all data_features into a single numpy array
X = []
Y = []
rows = []
for i in range(N_items):

    X_tmp = np.load(path_out+data_features[i])
    Y_tmp = np.load(path_out+data_votes[i])
    # if axis = -1 contains a nan then drop that row
    for r in np.where(np.isnan(X_tmp))[0]:
        rows.append(r)
rows = np.unique(rows)

for i in range(N_items):
    # exclude rows with nan
    X_tmp = np.load(path_out+data_features[i])
    Y_tmp = np.load(path_out+data_votes[i])
    X_tmp = np.delete(X_tmp, rows, axis = 0)
    X_tmp = X_tmp.reshape(1, *X_tmp.shape)
    Y_tmp = np.delete(Y_tmp, rows, axis = 0)
    Y_tmp = Y_tmp.reshape(1, *Y_tmp.shape)
    

    X.append(X_tmp)
    Y.append(Y_tmp)

X = np.vstack(X)
X= np.exp(X)
Y = np.vstack(Y)
Y = Y[0,:,:].reshape(1, *Y.shape[1:])

N_samples = X.shape[1]

In [ ]:
# kl divergence taking into account zeros
def kl(p, q):
    p = np.maximum(p, 1e-12)
    q = np.maximum(q, 1e-12)
    return np.mean(p * np.log(p / q))

def mean_KL(Y_p, X_1, N_samples, N_classes):
    # apply kl divergence to all time points
    KL = np.zeros((N_samples, N_classes))
    for i in range(N_samples):
        KL[i] = kl(Y_p[0,i,:], X_1[i,:])

    KL = np.sum(KL, axis=1)

    return np.mean(KL)

In [ ]:
Y_p = Y/np.sum(Y, axis=2).reshape(1,N_samples,1)

theta = [1/N_items]*N_items
theta_0 = np.array(theta).reshape(N_items,1,1)
X_0 = np.sum(theta_0*X, axis=0) 
X_0 = X_0/X_0.sum(axis=-1).reshape(N_samples, 1)

print(f"{mean_KL(Y_p, X_0, N_samples, N_classes):.2f}")

KL = []
lr = 0.001
for _ in range(150):

    grad = (np.sum(np.sum((Y_p*X)/np.sum(theta_0*X, axis=0).reshape(1,N_samples,6), axis = -1), axis = -1) -1)#
    theta_new = (theta_0.reshape(N_items) - lr*grad).reshape(N_items,1,1)
    
    X_0 = np.sum(theta_new*X, axis=0) 
    X_0 = X_0/X_0.sum(axis=-1).reshape(N_samples, 1)
    KL.append(mean_KL(Y_p, X_0, N_samples, N_classes))
    if len(KL)>1:
        if KL[-1]>KL[-2]:
            break
        else:
            theta_0 = theta_new

    # break

print(f"{mean_KL(Y_p, X_0, N_samples, N_classes):.2f}")
# plot the KL divergence
plt.plot(KL, '.')

In [ ]:
# load all data_features into a single numpy array
X_test = []
Y_test = []
rows = []
for i in range(N_items):

    X_tmp = np.load(path_out+data_features[i])
    Y_tmp = np.load(path_out+data_votes[i])
    # if axis = -1 contains a nan then drop that row
    for r in np.where(np.isnan(X_tmp))[0]:
        rows.append(r)
rows = np.unique(rows)
print(rows)

for i in range(N_test_items):
    # exclude rows with nan
    X_tmp = np.load(path_out+data_features[i])
    Y_tmp = np.load(path_out+data_votes[i])
    X_tmp = np.delete(X_tmp, rows, axis = 0)
    X_tmp = X_tmp.reshape(1, *X_tmp.shape)
    Y_tmp = np.delete(Y_tmp, rows, axis = 0)
    Y_tmp = Y_tmp.reshape(1, *Y_tmp.shape)
    

    X_test.append(X_tmp)
    Y_test.append(Y_tmp)

X_test = np.vstack(X_test)
X_test= np.exp(X_test)
Y_test = np.vstack(Y_test)
Y_test = Y[0,:,:].reshape(1, *Y_test.shape[1:])

N_test_samples = X_test.shape[1]

In [ ]:
X_ensemble = np.sum(theta_new*X_test, axis=0)
X_ensemble = X_ensemble/X_ensemble.sum(axis=-1).reshape(N_test_samples, 1)
Y_p_test = Y_test/np.sum(Y_test, axis=2).reshape(1,N_test_samples,1)

KL_test = mean_KL(Y_p_test, X_ensemble, N_test_samples, N_classes)

print(f"{KL_test:.2f}")

In [ ]:
from sklearn.metrics import accuracy_score

y_test_class = np.argmax(Y_p_test, axis=2)[0,:]
predictions = np.argmax(X_ensemble, axis=1)

accuracy = accuracy_score(y_test_class, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

cm = confusion_matrix(y_test_class, predictions)

plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')
plt.xticks(ticks=np.arange(6) + 0.5, labels=votes_cols)
plt.yticks(ticks=np.arange(6) + 0.5, labels=votes_cols)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix Ensemble model')
plt.show()

cm_p = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 10))
sns.heatmap(cm_p, annot=True, fmt=".1f", cmap='Blues')
plt.xticks(ticks=np.arange(6) + 0.5, labels=votes_cols)
plt.yticks(ticks=np.arange(6) + 0.5, labels=votes_cols)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix Ensemble model')
plt.show()

### Delete data after run to save space

In [ ]:
# permanently delete all files and folders in the train and valid tmp data folder
# for folder in ["train/", "valid/"]:
#     for file in os.listdir(save_path + folder):
#         os.remove(save_path + folder + file)